# Open AI मॉडल्स का फाइन ट्यूनिंग

यह नोटबुक Open AI की [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) डाक्यूमेंटेशन में दी गई मौजूदा गाइडेंस पर आधारित है।

फाइन-ट्यूनिंग आपके एप्लिकेशन के लिए फाउंडेशन मॉडल्स की परफॉर्मेंस को बेहतर बनाता है, क्योंकि इसमें मॉडल को उस खास उपयोग या परिदृश्य से जुड़े अतिरिक्त डेटा और संदर्भ के साथ फिर से ट्रेन किया जाता है। ध्यान दें कि _few shot learning_ और _retrieval augmented generation_ जैसी प्रॉम्प्ट इंजीनियरिंग तकनीकों से आप डिफ़ॉल्ट प्रॉम्प्ट में प्रासंगिक डेटा जोड़कर क्वालिटी बढ़ा सकते हैं। हालांकि, इन तरीकों की सीमा टार्गेटेड फाउंडेशन मॉडल के अधिकतम टोकन विंडो साइज तक ही है।

फाइन-ट्यूनिंग के साथ, हम वास्तव में मॉडल को आवश्यक डेटा के साथ फिर से ट्रेन कर रहे हैं (जिससे हम अधिक उदाहरणों का उपयोग कर सकते हैं, जितना कि अधिकतम टोकन विंडो में आ सकता है) - और एक _कस्टम_ वर्शन का मॉडल डिप्लॉय कर रहे हैं, जिसे अब इनफरेंस के समय उदाहरण देने की जरूरत नहीं होती। इससे न केवल हमारे प्रॉम्प्ट डिज़ाइन की प्रभावशीलता बढ़ती है (हम टोकन विंडो का उपयोग अन्य चीजों के लिए अधिक लचीलापन के साथ कर सकते हैं) बल्कि संभावित रूप से हमारी लागत भी कम होती है (क्योंकि इनफरेंस के समय हमें मॉडल को भेजे जाने वाले टोकन की संख्या घट जाती है)।

फाइन ट्यूनिंग के 4 चरण होते हैं:
1. ट्रेनिंग डेटा तैयार करें और अपलोड करें।
1. ट्रेनिंग जॉब चलाएं और फाइन-ट्यून किया हुआ मॉडल प्राप्त करें।
1. फाइन-ट्यून मॉडल का मूल्यांकन करें और क्वालिटी के लिए दोहराएं।
1. जब संतुष्ट हों, तो फाइन-ट्यून मॉडल को इनफरेंस के लिए डिप्लॉय करें।

ध्यान दें कि सभी फाउंडेशन मॉडल्स फाइन-ट्यूनिंग को सपोर्ट नहीं करते - [OpenAI डाक्यूमेंटेशन देखें](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) ताजा जानकारी के लिए। आप पहले से फाइन-ट्यून किए गए मॉडल को भी फिर से फाइन-ट्यून कर सकते हैं। इस ट्यूटोरियल में, हम `gpt-35-turbo` को फाइन-ट्यूनिंग के लिए अपना टार्गेट फाउंडेशन मॉडल बनाएंगे।

---


### चरण 1.1: अपना डेटासेट तैयार करें

आइए एक चैटबॉट बनाते हैं जो आपको तत्वों की आवर्त सारणी को समझने में मदद करता है, और किसी तत्व के बारे में सवाल पूछने पर एक लिरिक के रूप में जवाब देता है। इस आसान ट्यूटोरियल में, हम सिर्फ एक डेटासेट बनाएंगे जिसमें कुछ उदाहरण जवाब होंगे, ताकि मॉडल को ट्रेन किया जा सके और डेटा का अपेक्षित फॉर्मेट दिखाया जा सके। असली दुनिया के उपयोग में, आपको बहुत सारे उदाहरणों के साथ एक बड़ा डेटासेट बनाना होगा। अगर आपके एप्लिकेशन डोमेन के लिए कोई ओपन डेटासेट उपलब्ध है, तो आप उसे भी इस्तेमाल कर सकते हैं और फाइन-ट्यूनिंग के लिए उसका फॉर्मेट बदल सकते हैं।

चूंकि हम `gpt-35-turbo` पर ध्यान दे रहे हैं और एक सिंगल-टर्न जवाब (chat completion) चाहते हैं, तो हम [इस सुझाए गए फॉर्मेट](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) का इस्तेमाल कर सकते हैं, जो OpenAI चैट कंप्लीशन की जरूरतों को दर्शाता है। अगर आपको मल्टी-टर्न बातचीत की जरूरत है, तो आप [मल्टी-टर्न उदाहरण फॉर्मेट](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) का इस्तेमाल करेंगे, जिसमें `weight` पैरामीटर होता है, जिससे यह तय होता है कि कौन से मैसेज फाइन-ट्यूनिंग में शामिल किए जाएं या नहीं।

हम यहां ट्यूटोरियल के लिए आसान सिंगल-टर्न फॉर्मेट का इस्तेमाल करेंगे। डेटा [jsonl फॉर्मेट](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) में है, जिसमें हर लाइन पर एक रिकॉर्ड होता है, और हर रिकॉर्ड JSON ऑब्जेक्ट के रूप में लिखा जाता है। नीचे दिए गए स्निपेट में 2 रिकॉर्ड्स का उदाहरण है - पूरा सैंपल सेट (10 उदाहरण) के लिए [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) देखें, जिसे हम अपने फाइन-ट्यूनिंग ट्यूटोरियल में इस्तेमाल करेंगे। **नोट:** हर रिकॉर्ड _एक ही लाइन_ में लिखा होना चाहिए (जैसा कि फॉर्मेटेड JSON फाइल में आमतौर पर लाइनों में बांटा जाता है, वैसे नहीं)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

असली दुनिया के उपयोग में अच्छे नतीजों के लिए आपको बहुत ज्यादा उदाहरणों की जरूरत होगी - इसमें जवाबों की गुणवत्ता और फाइन-ट्यूनिंग के समय/खर्च के बीच संतुलन बनाना होगा। हम यहां छोटा सेट इस्तेमाल कर रहे हैं ताकि प्रक्रिया जल्दी पूरी हो सके और समझाया जा सके। ज्यादा जटिल फाइन-ट्यूनिंग ट्यूटोरियल के लिए [यह OpenAI Cookbook उदाहरण](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) देखें।


### चरण 1.2 अपना डेटासेट अपलोड करें

डेटा को Files API का उपयोग करके [यहाँ बताए गए तरीके से](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) अपलोड करें। ध्यान दें कि इस कोड को चलाने से पहले आपको ये कदम पूरे करने होंगे:
 - `openai` Python पैकेज इंस्टॉल किया हो (नवीनतम फीचर्स के लिए सुनिश्चित करें कि आप वर्शन >=0.28.0 का उपयोग कर रहे हैं)
 - अपने OpenAI API key को `OPENAI_API_KEY` एनवायरनमेंट वेरिएबल में सेट किया हो
अधिक जानने के लिए, कोर्स के लिए दिया गया [सेटअप गाइड](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) देखें।

अब, अपने लोकल JSONL फाइल से अपलोड के लिए एक फाइल बनाने के लिए कोड चलाएँ।


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### चरण 2.1: SDK के साथ फाइन-ट्यूनिंग जॉब बनाएं


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### चरण 2.2: जॉब की स्थिति जांचें

यहाँ कुछ चीज़ें हैं जो आप `client.fine_tuning.jobs` API के साथ कर सकते हैं:
- `client.fine_tuning.jobs.list(limit=<n>)` - पिछले n फाइन-ट्यूनिंग जॉब्स की सूची देखें
- `client.fine_tuning.jobs.retrieve(<job_id>)` - किसी विशेष फाइन-ट्यूनिंग जॉब का विवरण प्राप्त करें
- `client.fine_tuning.jobs.cancel(<job_id>)` - किसी फाइन-ट्यूनिंग जॉब को रद्द करें
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - जॉब से n तक की घटनाओं की सूची देखें
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

इस प्रक्रिया का पहला कदम है _ट्रेनिंग फाइल को वैलिडेट करना_ ताकि यह सुनिश्चित हो सके कि डेटा सही फॉर्मेट में है।


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### चरण 2.3: प्रगति पर नज़र रखने के लिए इवेंट्स ट्रैक करें


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### चरण 2.4: OpenAI डैशबोर्ड में स्थिति देखें


आप OpenAI वेबसाइट पर जाकर और प्लेटफ़ॉर्म के _Fine-tuning_ सेक्शन में जाकर भी स्थिति देख सकते हैं। यहां आपको मौजूदा जॉब की स्थिति दिखेगी, साथ ही आप पहले के जॉब निष्पादन रन का इतिहास भी देख सकते हैं। इस स्क्रीनशॉट में, आप देख सकते हैं कि पिछला निष्पादन असफल रहा था, और दूसरी बार चलाने पर सफलता मिली। संदर्भ के लिए, ऐसा तब हुआ जब पहली बार रन में गलत फॉर्मेट वाले रिकॉर्ड्स के साथ JSON फ़ाइल का उपयोग किया गया था - जब इसे सही किया गया, तो दूसरी बार रन सफलतापूर्वक पूरा हुआ और मॉडल उपयोग के लिए उपलब्ध हो गया।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.hi.png)


आप नीचे दिखाए गए विज़ुअल डैशबोर्ड में और नीचे स्क्रॉल करके स्टेटस संदेश और मेट्रिक्स भी देख सकते हैं:

| संदेश | मेट्रिक्स |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.hi.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.hi.png)|


### चरण 3.1: आईडी प्राप्त करें और कोड में फाइन-ट्यून किए गए मॉडल का परीक्षण करें


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### चरण 3.2: फाइन-ट्यून किए गए मॉडल को प्लेग्राउंड में लोड करें और टेस्ट करें

अब आप फाइन-ट्यून किए गए मॉडल को दो तरीकों से टेस्ट कर सकते हैं। पहला, आप Playground पर जाएं और Models ड्रॉप-डाउन से अपनी नई फाइन-ट्यून की गई मॉडल को चुनें। दूसरा विकल्प है Fine-tuning पैनल में दिख रहे "Playground" विकल्प का उपयोग करना (ऊपर दिए गए स्क्रीनशॉट में देखें), जिससे एक _तुलनात्मक_ व्यू खुलता है जिसमें फाउंडेशन और फाइन-ट्यून किए गए मॉडल दोनों को साइड-बाय-साइड दिखाया जाता है ताकि आप जल्दी से तुलना कर सकें।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.hi.png)

सिर्फ वही सिस्टम कॉन्टेक्स्ट भरें जो आपने अपने ट्रेनिंग डेटा में इस्तेमाल किया था और अपना टेस्ट सवाल डालें। आप देखेंगे कि दोनों तरफ एक जैसा कॉन्टेक्स्ट और सवाल अपडेट हो जाता है। तुलना चलाएं और आप देखेंगे कि दोनों के आउटपुट में क्या फर्क है। _ध्यान दें कि फाइन-ट्यून किया गया मॉडल आपके उदाहरणों में दिए गए फॉर्मेट में जवाब देता है, जबकि फाउंडेशन मॉडल सिर्फ सिस्टम प्रॉम्प्ट को फॉलो करता है_।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.hi.png)

आप देखेंगे कि तुलना में हर मॉडल के लिए टोकन काउंट और इंफरेंस में लगा समय भी दिखाया जाता है। **यह खास उदाहरण सिर्फ प्रक्रिया समझाने के लिए है, असल दुनिया के डेटा या सीनारियो को नहीं दर्शाता**। आप देख सकते हैं कि दोनों सैंपल्स में टोकन की संख्या एक जैसी है (क्योंकि सिस्टम कॉन्टेक्स्ट और यूजर प्रॉम्प्ट एक जैसे हैं), लेकिन फाइन-ट्यून किए गए मॉडल (कस्टम मॉडल) को इंफरेंस में ज्यादा समय लगता है।

असल दुनिया के मामलों में, आप ऐसा सिंपल उदाहरण नहीं लेंगे, बल्कि असली डेटा (जैसे कस्टमर सर्विस के लिए प्रोडक्ट कैटलॉग) पर फाइन-ट्यूनिंग करेंगे, जहां जवाब की क्वालिटी ज्यादा साफ नजर आएगी। _ऐसे_ मामलों में, फाउंडेशन मॉडल से उतनी ही क्वालिटी का जवाब पाने के लिए आपको ज्यादा कस्टम प्रॉम्प्ट इंजीनियरिंग करनी होगी, जिससे टोकन की खपत और इंफरेंस का समय दोनों बढ़ सकते हैं। _इसे आज़माने के लिए, OpenAI Cookbook में दिए गए फाइन-ट्यूनिंग उदाहरण देखें और शुरुआत करें_।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या गलतियाँ हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
